<a href="https://colab.research.google.com/github/AbeHandler/AbeHandler.github.io/blob/master/s20240508.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import pandas as pd
from random import shuffle
from datasets import Dataset, DatasetDict

from sklearn.model_selection import train_test_split

# Load the data from a JSONL file
with open("sentences.may8.jsonl", "r") as file:
    data = [json.loads(line) for line in file]

# Shuffle the data
shuffle(data)

# Convert the list of dictionaries to a DataFrame
data_df = pd.DataFrame(data)

# Preprocess the text by converting it to lowercase
data_df["text"] = data_df["sentence"].apply(lambda x: x.lower())

# Drop the original 'sentence' column as it's no longer needed
data_df.drop(columns=["sentence"], inplace=True)

# Split the data into training and test sets
train_data, test_data = train_test_split(data_df, test_size=0.2)

# Convert the train and test DataFrame to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Create a DatasetDict to hold both the train and test datasets
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [5]:
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=32,
    num_epochs=4,
)

In [7]:
from setfit import Trainer
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_dict["train"],
)


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/339 [00:00<?, ? examples/s]

In [8]:
trainer.train()

***** Running training *****
  Num unique pairs = 57860
  Batch size = 32
  Num epochs = 4
  Total optimization steps = 7236


Step,Training Loss


In [9]:
trainer.evaluate(dataset_dict["test"])

***** Running evaluation *****


{'accuracy': 0.8117647058823529}

In [17]:
model.predict_proba(["CEO personality affects dynamic capability"])

tensor([[0.0219, 0.9781]], dtype=torch.float64)

In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
model.push_to_hub("abehandlerorg/setfit")
model = SetFitModel.from_pretrained("abehandlerorg/setfit")

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config_setfit.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

In [1]:
! pip install setfit
! pip install torch --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 47.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (73